In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4"  # define device ids to use before importing torch

import torch
import random
import numpy as np

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

In [2]:
from requests import get

# download files for sentiment classification
def download(url, filename):
    with open(filename, "wb") as file:
        response = get(url)
        file.write(response.content)

download("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", "ratings_train.txt")
download("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", "ratings_test.txt")

with open("ratings_train.txt", "r") as file:
    for i in range(5):
        print(file.readline())

with open("ratings_train.txt", "r", encoding="utf-8") as file:
    contents = file.read()
    lines = contents.split("\n")[1:]
    train_data = [line.split("\t") for line in lines if len(line) > 0]

with open("ratings_test.txt", "r", encoding="utf-8") as file:
    contents = file.read()
    lines = contents.split("\n")[1:]
    test_data = [line.split("\t") for line in lines if len(line) > 0]

id	document	label

9976970	아 더빙.. 진짜 짜증나네요 목소리	0

3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1

10265843	너무재밓었다그래서보는것을추천한다	0

9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0



In [3]:
import re

tokenizer = lambda x: x.split()

tokenized_train_dataset = []
tokenized_test_dataset = []

# remove unnecessary chracaters
for data in train_data:
    text = re.sub(r'[,.!?;:()\"\'-]', ' ', data[1])
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', ' ', text)
    tokens = tokenizer(text)
    labels = data[2]
    tokenized_train_dataset.append((tokens, labels))

for data in test_data:
    text = re.sub(r'[,.!?;:()\"\'-]', ' ', data[1])
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', ' ', text)
    tokens = tokenizer(text)
    labels = data[2]
    tokenized_test_dataset.append((tokens, labels))

In [4]:
from collections import Counter

token_counter = Counter()

for tokens, _ in tokenized_train_dataset:
    token_counter.update(tokens)

# remove tokens that appear only twice or less
min_count = 2
cleaned_vocab = {"[PAD]":0, "[UNK]":1}
cleaned_vocab_idx = 2

for token, count in token_counter.items():
    if count > min_count:
        cleaned_vocab[token] = cleaned_vocab_idx
        cleaned_vocab_idx += 1

In [5]:
# make word2vec train data
word2vec_train_datas = []
for train_text, _ in tokenized_train_dataset:
    word2vec_train_datas.append([word for word in train_text])

word2vec_train_datas[:5]

[['아', '더빙', '진짜', '짜증나네요', '목소리'],
 ['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나'],
 ['너무재밓었다그래서보는것을추천한다'],
 ['교도소', '이야기구먼', '솔직히', '재미는', '없다', '평점', '조정'],
 ['사이몬페그의',
  '익살스런',
  '연기가',
  '돋보였던',
  '영화',
  '스파이더맨에서',
  '늙어보이기만',
  '했던',
  '커스틴',
  '던스트가',
  '너무나도',
  '이뻐보였다']]

In [6]:
from gensim.models import Word2Vec

# call CBOW or SkipGram
# CBOW_W2V = Word2Vec(sentences = word2vec_train_datas, vector_size = 32, window = 5, min_count = 1, workers = 4, sg = 0)
SkipGram_W2V = Word2Vec(sentences = word2vec_train_datas, vector_size = 32, window = 5, min_count = 1, workers = 4, sg = 1)

In [7]:
import numpy as np

# make embedding lookup matrix
embedding_list = []

for token, idx in cleaned_vocab.items():
    if token in SkipGram_W2V.wv:
        embedding_list.append(SkipGram_W2V.wv[token])
    elif token == "[PAD]":
        embedding_list.append(np.zeros(SkipGram_W2V.wv.vectors.shape[1]))
    elif token == "[UNK]":
        embedding_list.append(np.random.uniform(-1, 1, SkipGram_W2V.wv.vectors.shape[1]))
    else:
        embedding_list.append(np.random.uniform(-1, 1, SkipGram_W2V.wv.vectors.shape[1]))

embedding_loopup_matrix = np.vstack(embedding_list)

print(embedding_loopup_matrix.shape)    # (43354, 32)
print(len(cleaned_vocab))               # 43354

(43354, 32)
43354


In [8]:
from torch.utils.data import Dataset, DataLoader

# define dataset class
class SentimentDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label = int(self.data[index][1])
        tokens = self.data[index][0]

        token_ids = [self.vocab[token] if token in self.vocab else 1 for token in tokens]
        
        # Text는 데이터 별로 길이가 다르다.
        # 100은 sequence의 길이를 임의로 설정한 것
        # 100 token보다 길이가 긴 것은 100 token으로 자르기!
        # 100 token보다 길이가 짧은 것은 PAD token을 뒤에 추가!
        # --> 길이가 같은 vector를 모아 matrix를 만들기 위함! (GPU 연산을 위해)
        if len(token_ids) > 100:
            token_ids = token_ids[:100]
        else:
            token_ids = token_ids[:100] + [0] * (100 - len(token_ids))

        return torch.tensor(token_ids), torch.tensor(label)

In [9]:
import torch.nn as nn
import lightning as pl

class SentimentClassifierPL(pl.LightningModule):
    def __init__(self, sentiment_classifier):
        super(SentimentClassifierPL, self).__init__()
        self.model = sentiment_classifier
        self.loss = nn.CrossEntropyLoss()
        
        self.validation_step_outputs = []
        self.test_step_outputs = []
        self.save_hyperparameters()
    
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.loss(outputs, labels)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.loss(outputs, labels)
        self.log("val_loss", loss)
        self.validation_step_outputs.append((loss, outputs, labels))
        return loss, outputs, labels
    
    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        
        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("val_accuracy", accuracy)
        self.validation_step_outputs.clear()
    
    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.loss(outputs, labels)
        self.log("test_loss", loss)
        self.test_step_outputs.append((loss, outputs, labels))
        return loss, outputs, labels
    
    def on_test_epoch_end(self):
        outputs = self.test_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_test_loss", avg_loss)
        
        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("test_accuracy", accuracy)
        self.test_step_outputs.clear()
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelSummary

wandb.login()

def check_vocab_properties(vocab):
    print(f"Vocab size: {len(vocab)}")
    print(f"Vocab items: {list(vocab.items())[:5]}")


def check_performance(model, vocab,train_data, test_data, wandb_log_name):
    wandb_logger = WandbLogger(project="NLP", name=wandb_log_name, group="Lec04")

    pl_model = SentimentClassifierPL(model)

    train_dataset = SentimentDataset(train_data, vocab)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_dataset = SentimentDataset(test_data, vocab)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    test_dataset = SentimentDataset(test_data, vocab)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

    trainer = pl.Trainer(
        max_epochs=3,
        accelerator="gpu",
        logger=wandb_logger,
        callbacks=[ModelSummary(max_depth=2)]
    )

    trainer.fit(
        model=pl_model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )

    trainer.test(dataloaders=test_loader)

    wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: noeyhesx. Use `wandb login --relogin` to force relogin


In [14]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, vocab_size):
        super(MLP, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_loopup_matrix), freeze=False)
        self.fc1 = nn.Linear(32 * 100, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        print(x.size())
        x = self.embedding(x)
        print(x.size())
        x = x.view(-1, 32 * 100)
        print(x.size())
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size):
        super(TextCNN, self).__init__()
        self.SG_embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_loopup_matrix), freeze=True)
        self.RD_embedding = nn.Embedding(vocab_size, 32)
        
        self.SG_conv1 = nn.Conv2d(1, 32, (3, 32))
        self.SG_conv2 = nn.Conv2d(1, 32, (4, 32))
        self.SG_conv3 = nn.Conv2d(1, 32, (5, 32))
        
        self.RD_conv1 = nn.Conv2d(1, 32, (3, 32))
        self.RD_conv2 = nn.Conv2d(1, 32, (4, 32))
        self.RD_conv3 = nn.Conv2d(1, 32, (5, 32))
        
        self.fc = nn.Linear(6*32, 2)

        
    def forward(self, x):
        SG_embedding = self.SG_embedding(x).unsqueeze(1)
        print(SG_embedding.size())
        RD_embedding = self.RD_embedding(x).unsqueeze(1)

        SG_conv1_feature = F.relu(self.SG_conv1(SG_embedding).squeeze(3))
        print(SG_conv1_feature.size())
        SG_conv2_feature = F.relu(self.SG_conv2(SG_embedding).squeeze(3))
        print(SG_conv2_feature.size())
        SG_conv3_feature = F.relu(self.SG_conv3(SG_embedding).squeeze(3))
                                                                         
        RD_conv1_feature = F.relu(self.RD_conv1(RD_embedding).squeeze(3))
        RD_conv2_feature = F.relu(self.RD_conv2(RD_embedding).squeeze(3))
        RD_conv3_feature = F.relu(self.RD_conv3(RD_embedding).squeeze(3))

        SG_max1 = F.max_pool1d(SG_conv1_feature, SG_conv1_feature.size(2)).squeeze(2)
        print(SG_max1.size())
        SG_max2 = F.max_pool1d(SG_conv2_feature, SG_conv2_feature.size(2)).squeeze(2)  
        SG_max3 = F.max_pool1d(SG_conv3_feature, SG_conv3_feature.size(2)).squeeze(2)  

        RD_max1 = F.max_pool1d(RD_conv1_feature, RD_conv1_feature.size(2)).squeeze(2)  
        RD_max2 = F.max_pool1d(RD_conv2_feature, RD_conv2_feature.size(2)).squeeze(2)  
        RD_max3 = F.max_pool1d(RD_conv3_feature, RD_conv3_feature.size(2)).squeeze(2)  

        x = torch.cat([SG_max1, SG_max2, SG_max3, RD_max1, RD_max2, RD_max3], dim=1)
        print(x.size())

        x = self.fc(x)

        return x

In [30]:
class LSTM(nn.Module):
    def __init__(self, vocab_size):
        super(LSTM, self).__init__()
        hidden_size = 32
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_loopup_matrix), freeze=False)
        self.rnn = nn.LSTM(32, 32, batch_first=True, num_layers=2, bidirectional=False)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 2)
        )

    def forward(self, x):
        x = self.embedding(x)
        print(x.size())
        x, _ = self.rnn(x)
        print(x.size())
        x = x.mean(dim=1)
        print(x.size())
        # x = x[:, -1, :]
        x = self.fc(x)
        return x

In [12]:
class biLSTM(nn.Module):
    def __init__(self, vocab_size):
        super(biLSTM, self).__init__()
        hidden_size = 32
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_loopup_matrix), freeze=False)
        self.rnn = nn.LSTM(32, 32, batch_first=True, num_layers=2, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size * 2),
            nn.ReLU(),
            nn.Linear(hidden_size * 2, 2)
        )

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = x.mean(dim=1)
        # x = x[:, -1, :]
        x = self.fc(x)
        return x

In [31]:
mlp_model = MLP(len(cleaned_vocab))
textcnn_model = TextCNN(len(cleaned_vocab))
lstm_model = LSTM(len(cleaned_vocab))
bilstm_model = biLSTM(len(cleaned_vocab))

In [25]:
sample_inputs = torch.randint(0, len(cleaned_vocab), (2, 100))

In [22]:
mlp_model.forward(sample_inputs)

torch.Size([2, 100])
torch.Size([2, 100, 32])
torch.Size([2, 3200])


tensor([[-0.1286,  0.1403],
        [-0.0881,  0.1116]], grad_fn=<AddmmBackward0>)

In [18]:
textcnn_model.forward(sample_inputs)

torch.Size([2, 1, 100, 32])
torch.Size([2, 32, 98])
torch.Size([2, 32, 97])
torch.Size([2, 32])
torch.Size([2, 192])


tensor([[0.9387, 0.3004],
        [1.2668, 0.2256]], grad_fn=<AddmmBackward0>)

In [32]:
lstm_model.forward(sample_inputs)

torch.Size([2, 100, 32])
torch.Size([2, 100, 32])
torch.Size([2, 32])


tensor([[-0.0004, -0.0427],
        [-0.0006, -0.0425]], grad_fn=<AddmmBackward0>)

In [20]:
bilstm_model.forward(sample_inputs)

tensor([[0.0201, 0.0343],
        [0.0200, 0.0347]], grad_fn=<AddmmBackward0>)

In [21]:
check_performance(mlp_model, cleaned_vocab, tokenized_train_dataset, tokenized_test_dataset, "mlp")

/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'sentiment_classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['sentiment_classifier'])`.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | model           | MLP              | 1.7 M 
1 | model.embedding | Embedding        | 1.4 M 
2 | model.fc1       | Linear           | 320 K 
3 | model.fc2       | Linear           | 202   
4 | loss            | CrossEntropyLoss | 0     
-----------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.831     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 2344/2344 [00:06<00:00, 335.60it/s, v_num=kczm]  

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 2344/2344 [00:07<00:00, 333.97it/s, v_num=kczm]


/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at ./NLP/eukokczm/checkpoints/epoch=2-step=7032.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./NLP/eukokczm/checkpoints/epoch=2-step=7032.ckpt


Testing DataLoader 0: 100%|██████████| 782/782 [00:01<00:00, 730.95it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      avg_test_loss         0.4086398482322693
      test_accuracy         0.8113200068473816
        test_loss           0.40860047936439514
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


avg_test_loss,▁
avg_val_loss,▁▁█
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆▆▆▆█
test_accuracy,▁
test_loss,▁
train_loss,█▄▆▄▃▅▆▅▅▄▄▃▄▄▄▃▃▃▄▂▂▄▃▃▆▄▄▁▃▃▃▄▃▂▁▂▂▂▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▅█▁
val_loss,▁▁█
avg_test_loss,0.40864
avg_val_loss,0.40864


In [22]:
check_performance(textcnn_model, cleaned_vocab, tokenized_train_dataset, tokenized_test_dataset, "textcnn")

/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'sentiment_classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['sentiment_classifier'])`.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type             | Params
---------------------------------------------------------
0  | model              | TextCNN          | 2.8 M 
1  | model.SG_embedding | Embedding        | 1.4 M 
2  | model.RD_embedding | Embedding        | 1.4 M 
3  | model.SG_conv1     | Conv2d           | 3.1 K 
4  | model.SG_conv2     | Conv2d           | 4.1 K 
5  | model.SG_conv3     | Conv2d           | 5.2 K 
6  | model.RD_conv1     | Conv2d           | 3.1 K 
7  | model.RD_conv2     | Conv2d           | 4.1 K 
8  | model.RD_conv3     | Conv2d           | 5.2 K 
9  | model.fc           | Linear           | 386   
10 | loss               | CrossEntropyLoss | 0     
---------------------------------------------------------
1.4 M     Trainable params
1.4 M     Non-trainable params
2.8 M     Total params
11.199    Total estimated model params size (MB)


Epoch 2: 100%|██████████| 2344/2344 [00:08<00:00, 263.15it/s, v_num=f41c]  

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 2344/2344 [00:08<00:00, 261.83it/s, v_num=f41c]


/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at ./NLP/3onpf41c/checkpoints/epoch=2-step=7032.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./NLP/3onpf41c/checkpoints/epoch=2-step=7032.ckpt


Testing DataLoader 0: 100%|██████████| 782/782 [00:01<00:00, 662.27it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      avg_test_loss         0.45426586270332336
      test_accuracy         0.7851199507713318
        test_loss           0.4540354013442993
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


avg_test_loss,▁
avg_val_loss,▆▁█
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆▆▆▆█
test_accuracy,▁
test_loss,▁
train_loss,▇▆▅▄▅▅▅▄▄▅▆▅█▄▅▄▃▃▄▅▅▄▇▅▄▃▄▅▄▄▅▃▆▃▄▅▅▃▃▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁▇█
val_loss,▆▁█
avg_test_loss,0.45427
avg_val_loss,0.45427


In [23]:
check_performance(lstm_model, cleaned_vocab, tokenized_train_dataset, tokenized_test_dataset, "lstm")

/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'sentiment_classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['sentiment_classifier'])`.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | model           | LSTM             | 1.4 M 
1 | model.embedding | Embedding        | 1.4 M 
2 | model.rnn       | LSTM             | 16.9 K
3 | model.fc        | Sequential       | 1.1 K 
4 | loss            | CrossEntropyLoss | 0     
-----------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.621     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 2344/2344 [00:08<00:00, 270.86it/s, v_num=azlo]  

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 2344/2344 [00:08<00:00, 269.88it/s, v_num=azlo]


/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at ./NLP/mey8azlo/checkpoints/epoch=2-step=7032.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./NLP/mey8azlo/checkpoints/epoch=2-step=7032.ckpt


Testing DataLoader 0: 100%|██████████| 782/782 [00:01<00:00, 653.67it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      avg_test_loss         0.39808112382888794
      test_accuracy         0.8160799741744995
        test_loss           0.39798110723495483
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


avg_test_loss,▁
avg_val_loss,█▁▄
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆▆▆▆█
test_accuracy,▁
test_loss,▁
train_loss,█▆▆████▇▇▇▆▄▆▃▃▄▂▂▃▃▃▃▃▂▃▄▃▁▂▄▂▂▂▃▂▄▂▄▅▃
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁█▇
val_loss,█▁▄
avg_test_loss,0.39808
avg_val_loss,0.39808


In [24]:
check_performance(bilstm_model, cleaned_vocab, tokenized_train_dataset, tokenized_test_dataset, "bilstm")

/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'sentiment_classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['sentiment_classifier'])`.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | model           | biLSTM           | 1.4 M 
1 | model.embedding | Embedding        | 1.4 M 
2 | model.rnn       | LSTM             | 42.0 K
3 | model.fc        | Sequential       | 4.3 K 
4 | loss            | CrossEntropyLoss | 0     
-----------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.734     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 2344/2344 [00:09<00:00, 253.96it/s, v_num=439z]   

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 2344/2344 [00:09<00:00, 253.11it/s, v_num=439z]


/home/dev/anaconda3/envs/nlp/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at ./NLP/aojg439z/checkpoints/epoch=2-step=7032.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./NLP/aojg439z/checkpoints/epoch=2-step=7032.ckpt


Testing DataLoader 0: 100%|██████████| 782/782 [00:01<00:00, 659.51it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      avg_test_loss         0.40379056334495544
      test_accuracy         0.8178199529647827
        test_loss           0.4034873843193054
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


avg_test_loss,▁
avg_val_loss,▇▁█
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆▆▆▆█
test_accuracy,▁
test_loss,▁
train_loss,▇▄▇▅▄▅▅▅▅▅▃▄▅▄▄▃▃▃▃▂▃▃▂▃▃▂▁▂▄▂▃▁▂▄▁▁▃█▂▅
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁██
val_loss,▇▁█
avg_test_loss,0.40379
avg_val_loss,0.40379
